# Correlation computation
This script takes data preprocessed by preprocess_data script and calculates Pearson and Mutual Information correlations.

In [ ]:
from collections import Counter
from math import log

from pyspark import SparkContext, SQLContext

In [ ]:
# Maximum is your nr_of_cores x 2 
# (or the number of logical processes).
# Picking more or less will change the running time 
# (more doesn't mean that it's going to be faster).
THREADS_TO_USE = 4

In [ ]:
SC = SparkContext(f'local[{THREADS_TO_USE}]')
print(SC.version)
SQL_CONTEXT = SQLContext(SC)

# Pearson Correlation

In [ ]:
def pearson_corr(vector_1, vector_2):
    v1_squared_sum = sum([x**2 for x in vector_1])
    v2_squared_sum = sum([x**2 for x in vector_2])
    numerator = sum([x*y for (x,y) in zip(vector_1, vector_2)])
    denominator = (v1_squared_sum**(1/2)) * (v2_squared_sum**(1/2))
    if denominator == 0: 
        return 0
    return numerator/denominator

# Mutual Information Correlation

In [ ]:
def mutual_information(vector_a, vector_b):
    n = len(vector_a)
    probas_a = Counter(vector_a)
    probas_b = Counter(vector_b)
    probas_ab = Counter(zip(vector_a, vector_b))
    for a in probas_a:
        probas_a[a] /= n
    for b in probas_b:
        probas_b[b] /= n
    for ab in probas_ab:
        probas_ab[ab] /= n
    
    mi = 0
    for x in probas_a:
        p_a = probas_a[x]
        for y in probas_b:
            p_b, p_ab = probas_b[y], probas_ab[(x,y)]
            if p_ab == 0: 
                continue
            mi += (p_ab)*(log(p_ab) - log(p_a*p_b))
    return mi

# Correlation

In [ ]:
def compare_groups(group_a, group_b, correlation_function):
    """ Each group is a list of tuples. Each tuple is 
    in a format of ((Stock, Value), idx).
    
    Parameters:
    -----------
    Stock
        String with the name of the stock.
    Value
        List with the values of that stock.
    idx
        Unique index of that stock.
    """
    print(group_a[0], group_b[0])
    
    if group_a[0] == group_b[0]:
        size_of_groups = len(group_a[1])
                             
        for i in range(size_of_groups):
            stock_a = group_a[1][i][0]
            for j in range(i+1, size_of_groups):
                stock_b = group_b[1][j][0]
                if stock_a[0].rsplit('-', 1)[0] != stock_b[0].rsplit('-', 1)[0]:
                    yield (
                        correlation_function(stock_a[1], stock_b[1]), 
                        (stock_a[0], stock_b[0]),
                    )
    else:
        for zip_tuple_a in group_a[1]:
            stock_a = zip_tuple_a[0]
            for zip_tuple_b in group_b[1]:
                stock_b = zip_tuple_b[0]
                if stock_a[0].rsplit('-', 1)[0] != stock_b[0].rsplit('-', 1)[0]:
                    yield (
                        correlation_function(stock_a[1], stock_b[1]), 
                        (stock_a[0], stock_b[0]),
                    )

In [ ]:
def compute_correlations(rdd, number_groups, correlation):
    """
    Parameters:
    -----------
    
    rdd
        RDD in a format of [(StockA, ValueA), (StockB, ValueB), ...]
    number_groups
        Number if batches to divide the data into.
        It's best to pick a number that's bigger than nr of threads 
        you are using, but not much more.
    """
    # Assign indices to each stock.
    rdd = rdd.zipWithIndex()
    # [((StockA, ValueA), idx=0), ((StockB, ValueB), idx=1), ...]
    
    # Divide stocks into buckets.
    rdd = rdd.groupBy(
        lambda zipTuple: zipTuple[1] % number_groups
    ).mapValues(tuple)
    # [(0, (zip tuples where idx%batch_size==0)), 
    #  (1, (zip tuples where idx%batch_size==1)), 
    # ...]
    
    # Create combinations of all possible buckets.
    rdd = rdd.cartesian(rdd)
    # [((0, (zip tuple)), (0, (zip tuple))), 
    #  ((0, (zip tuple)), (1, (zip tuple))), 
    #  ..., 
    #  ((1, (zip tuple)), (0, (zip tuple))),
    #  ...] 
    # So all combinations including the useless ones (1,0), (2,0), (2,1)...
    
    # If we have a correlation between buckets (1,2), then filter out (2,1)
    # We need to keep (0,0), (1,1), ..., because we want correlations within the group
    rdd = rdd.filter(lambda x: x[0][0] <= x[1][0])
    # [((0, (zip tuple)), (1, (zip tuple))),
    # ..., 
    # ((1, (zip tuple)), (2, (zip tuple))), 
    # ...]
    
    # We need to correlate every stock inside each 
    # (zip tuple)=((StockA, ValueB), idx).
    correlations = rdd.flatMap(
        lambda x: compare_groups(x[0], x[1], correlation)
    )

    return correlations.collect()

In [ ]:
def prepare_data(df, prefix, column):
    """
    Parameters:
    -----------
    df
        The dataframe that was loaded from pearson or MI
    prefix
        p or MI depending on which correlation.
    column
        Column name to use for correlation.
    """
    rdd = df.select(
        'stock', f'{prefix}_{column}'
    ).rdd.map(tuple).groupByKey().sortByKey()
    # [(StockA, ValuesA), (StockB, ValuesB)]
    
    rdd = rdd.map(lambda x: (f'{x[0]}-{column}', x[1]))
    # [(StockA_opening_price, ValuesA), (StockB_opening_price, ValuesB)]

    return rdd

# Calculate correlations

In [ ]:
def get_correlations(parquet_name, prefix, correlation_func):
    """ Gets correlations for specified data.
    
    Parameters:
    -----------
    parquet_name
        Name of the file were the data is located.
    prefix
        p or MI depending whether it's Pearson or Mutual information.
    correlation_func
        `pearson_corr` or `mutual_information`.
    """
    
    columns_for_corr = ['opening_price', 'highest_price', 'lowest_price']
    
    stock_data = SQL_CONTEXT.read.parquet(parquet_name)
    all_stocks_data = [
        prepare_data(stock_data, prefix, col) 
        for col 
        in columns_for_corr
    ]
    stock_rdd = SC.union(all_stocks_data).sortByKey()
    
    print(f'We have {stock_rdd.count()} vectors '
          f'with {len(stock_rdd.take(1)[0][1])} dimensions each.')
    
    # Because some workers will get less work it's good
    # to divide data into THREADS_TO_USE*x batches
    # instead if THREADS_TO_USE batches
    result = compute_correlations(
        stock_rdd, THREADS_TO_USE*2, correlation_func)
    
    result.sort(reverse=True, key=lambda tup: abs(tup[0]))
    return result[:10]

In [ ]:
%%time
pearson_correlated_data = get_correlations(
    'pearson_corr.parquet', 'p', pearson_corr,
)

print('Top 10 correlations according to Pearson: ')
for data_point in pearson_correlated_data:
    print(data_point)

In [ ]:
%%time
MI_correlated_data = get_correlations(
    'MI_corr.parquet', 'MI', mutual_information,
)

print('Top 10 correlations according to Mutual Information: ')
for data_point in MI_correlated_data:
    print(data_point)

In [ ]:
SC.stop()